In [1]:
import pandas as pd

In [2]:
data = pd.read_parquet('../data/data_made_restaurants.parquet')

In [3]:
customer_chain_pairs = data[['customer_id', 'chain_id']].drop_duplicates()

In [4]:
customer_chain_pairs.shape

(2203119, 2)

# Перевод айди в шкалу от нуля

In [5]:
def id_mapper(original_ids):
    mapping = dict()
    new_id = 0
    for original_id in original_ids:
        mapping[original_id] = new_id
        new_id+=1
    return mapping

In [6]:
customer_ids = sorted(customer_chain_pairs['customer_id'].unique())
customer_to_user_mapping = id_mapper(customer_ids)

chain_ids = sorted(customer_chain_pairs['chain_id'].unique())
chain_to_new_chain_mapping = id_mapper(chain_ids)

In [7]:
customer_chain_pairs['user_id'] = customer_chain_pairs['customer_id'].map(customer_to_user_mapping)
customer_chain_pairs['new_chain_id'] = customer_chain_pairs['chain_id'].map(chain_to_new_chain_mapping)

In [8]:
customer_chain_pairs.head()

,customer_id,chain_id,user_id,new_chain_id
0,15955880,140718,320220,5452
1,62512097,140718,683586,5452
2,64977556,140718,731183,5452
3,81281415,140718,1029017,5452
4,72045218,140718,848987,5452


# Сохраним новые айди

In [9]:
customer_chain_pairs.to_parquet('../data/preprocessed/user_id_chain_id.parquet')

# Создадим спарс матрицу

In [10]:
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix, lil_matrix

In [11]:
u_c_pairs = customer_chain_pairs[['user_id', 'new_chain_id']].to_numpy()

In [12]:
sparse_matrix_shape = (u_c_pairs[:, 0].max()+1, u_c_pairs[:, 1].max()+1)
sparse_matrix_shape

(1559890, 14148)

In [13]:
relation_matrix = lil_matrix(sparse_matrix_shape, dtype=np.int8)

In [14]:
for i, j in u_c_pairs:
    relation_matrix[i,j]=1
    #print(i,j)

In [15]:
orders_matrix = relation_matrix.tocsr()

In [16]:
orders_matrix.sum(axis=1)

matrix([[123],
        [  2],
        [  1],
        ...,
        [  1],
        [  1],
        [  1]])

In [17]:
import scipy
scipy.sparse.save_npz('../data/preprocessed/orders_marix.npz', orders_matrix)